In [1]:
import requests
import pandas as pd
import time

In [5]:
# IGDB API endpoint for games
url = 'https://api.igdb.com/v4/games'
client_id = 'sxiuj4y04yliue4d66h7dkoxt3sjjf'
access_token = 'ho99rm814hddmzvmzfedoftp2igx32'
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json'
}

In [13]:
def get_genres(game_titles):
    # Join the game titles into a string formatted for the query
    titles_query = ','.join(f'"{title}"' for title in game_titles)  # creates "title1","title2", etc.
    
    # Create the body using proper string formatting
    body = f'fields name, genres.name; where name=({titles_query});'
    
    # Make the request
    response = requests.post(url, headers=headers, data=body)
    
    # Check for a valid response
    if response.status_code == 200:
        games_data = response.json()
        # Create a dictionary with game names as keys and list of genre names as values
        genre_dict = {game['name']: [genre['name'] for genre in game.get('genres', [])] for game in games_data}
              
        return genre_dict
        
    else:
        print(f'Failed to retrieve data: {response.status_code}')
        return {}
    

In [19]:
# Example list of game titles
game_titles = ['Cities: Skylines', 'Counter-Strike: Source']

# Create a DataFrame
df = pd.DataFrame(game_titles, columns=['Game Title'])

genres_dict = get_genres(game_titles)
genres_dict

{'Cities: Skylines': ['Simulator', 'Strategy']}

In [100]:
titles_query = ','.join(f'"{title}"' for title in game_titles)
titles_query

'"The Elder Scrolls V: Skyrim","The Witcher","Dota 2","Cyberpunk 2077","God of War","Elden Ring"'

In [90]:
df

,Game Title
0,The Witcher
1,Dota 2
2,Cyberpunk 2077
3,God of War
4,Elden Ring
5,The Elder Scroll V: Skyrim


In [109]:
import time
import requests

def get_all_genres(game_titles):
    # Initialize variables to handle pagination
    limit = 500  # Set a high limit per page, adjust based on what the API supports
    offset = 0
    all_games_data = []

    while True:
        # Join the game titles into a string formatted for the query
        titles_query = ','.join(f'"{title}"' for title in game_titles)
        
        # Create the body using proper string formatting with pagination
        body = f'fields name, genres.name; where name=({titles_query}); limit {limit}; offset {offset};'
        
        # Make the request
        response = requests.post(url, headers=headers, data=body)
        
        # Check for a valid response
        if response.status_code == 200:
            games_data = response.json()
            all_games_data.extend(games_data)
            
            # Check if fewer results than the limit were returned; if so, break the loop
            if len(games_data) < limit:
                break

            # Increment the offset for the next page
            offset += limit
        else:
            print(f'Failed to retrieve data: {response.status_code}')
            break

        # Delay to respect API rate limit
        time.sleep(1)
    
    # Aggregate the results
    return {game['name']: [genre['name'] for genre in game.get('genres', [])] for game in all_games_data}


In [110]:
genres_dict = get_all_genres(game_titles)
genres_dict

{'God of War': ['Puzzle', "Hack and slash/Beat 'em up", 'Adventure'],
 'The Witcher': ['Role-playing (RPG)', 'Adventure'],
 'Dota 2': ['Strategy', 'MOBA'],
 'The Elder Scrolls V: Skyrim': ['Role-playing (RPG)', 'Adventure'],
 'Elden Ring': ['Role-playing (RPG)', 'Adventure'],
 'Cyberpunk 2077': ['Shooter', 'Role-playing (RPG)', 'Adventure']}

In [22]:
def search_game(title):
    url = 'https://api.igdb.com/v4/games'
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}'
    }
    # Simplify and normalize the title for the search query
    body = f'search "{title}"; fields name, genres.name; limit 1;'
    
    response = requests.post(url, headers=headers, data=body)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]['genres'][0]['name'] if 'genres' in data[0] and data[0]['genres'] else 'Genre not found'
        else:
            return 'No match found'
    else:
        print(f'API request failed with status code {response.status_code}')
        return 'API request failed'

In [25]:
print(search_game('Cities SkyLine'))

Simulator
